In [ ]:
#Algorithm 1. Stacked GRU Flight Fare Prediction Model.
#Step 1: Define the StackedGRU class
class StackedGRU(nn.Module):
    #Step 2: Define the constructor to initialize the model parameters
    def __init__(self, input_size, hidden_size, output_size, num_layers, dropout, window_size=1):
        super(StackedGRU, self).__init__()
        #Step 3: Set the model parameters
        self.input_size = input_size   # Size of input layer
        self.hidden_size = hidden_size # Size of hidden layers
        self.output_size = output_size # Size of output layer
        self.num_layers = num_layers   # Number of layers in the stacked GRU model
        self.dropout = dropout # Dropout probability
        self.window_size = window_size # Number of time steps to include in each window        
        #Step 4: Create a list of GRU layers
        self.gru_layers = nn.ModuleList()    
        #Step 5: Add the first GRU layer
        self.gru_layers.append(nn.GRU(input_size, hidden_size, batch_first=True, dropout=dropout))  
        #Step 6: Add additional GRU layers if num_layers > 1
        for i in range(num_layers-1):
            self.gru_layers.append(nn.GRU(hidden_size, hidden_size, batch_first=True, dropout=dropout))      
        #Step 7: Create the output layer
        self.linear = nn.Linear(hidden_size * window_size, output_size)  
    #Step 8: Define the forward pass through the stacked GRU model
    def forward(self, x):
        #Step 9: Reshape the input into windows of size self.window_size
        x = x.view(x.shape[0], -1, self.window_size, self.input_size)     
        #Step 10: Transpose the input to (batch_size, window_size, sequence_length, input_size)
        x = x.transpose(1, 2)        
        #Step 11: Flatten the input to (batch_size * window_size, sequence_length, input_size)
        x = x.contiguous().view(-1, x.shape[3], x.shape[2])       
        #Step 12: Pass the input through each GRU layer in the stacked model
        for i in range(self.num_layers):
            #Step 12a: Get the current GRU layer
            output, _ = self.gru_layers[i](x)           
            #Step 12b: Apply dropout to the output
            output = F.dropout(output, p=self.dropout, training=self.training)            
            #Step 12c: Set the input for the next layer to be the output of the current layer
            x = output        
        #Step 13: Reshape the output to (batch_size, window_size, hidden_size * sequence_length)
        output = output.contiguous().view(-1, self.window_size, output.shape[2] * output.shape[1])        
        #Step 14: Flatten the output to (batch_size * window_size, hidden_size * sequence_length)
        output = output.view(-1, output.shape[2])      
        #Step 15: Pass the final output through the output layer
        out = self.linear(output)      
         #Step 16: Reshape the output to (batch_size, window_size, output_size)
        out = out.view(-1, self.window_size, self.output_size)     